In [127]:
import matplotlib.pyplot as plt
import numpy as np
from tensorflow import keras

In [128]:
ROOT_URL = "https://raw.githubusercontent.com/hfawaz/cd-diagram/master/FordA/"

def read_data(filename):
    data = np.loadtxt(ROOT_URL + filename, delimiter="\t")
    y = data[:,0].astype(int)
    x = data[:,1:]
    
    return x,y


xtr, ytr = read_data("FordA_TRAIN.tsv")
xts, yts = read_data("FordA_TEST.tsv")

print(xtr.shape, ytr.shape, xts.shape, yts.shape)

classes = np.unique(np.concatenate((ytr, yts)))

# reshape into multivariate value
xtr = xtr.reshape((xtr.shape[0],xtr.shape[1],1))
xts = xts.reshape((xts.shape[0],xts.shape[1],1))
print(xtr.shape, xts.shape)

# shuffle
idx = np.random.permutation(xtr.shape[0])
xtr, ytr = xtr[idx], ytr[idx]

# labels tp positive ints
ytr[ytr == -1] = 0
yts[yts == -1] = 0

(3601, 500) (3601,) (1320, 500) (1320,)
(3601, 500, 1) (1320, 500, 1)


In [129]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    x = keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs, inputs)
    x = keras.layers.Dropout(dropout)(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)
    res = x + inputs

    x = keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(res)
    x = keras.layers.Dropout(dropout)(x)
    x = keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    x = keras.layers.LayerNormalization(epsilon=1e-6)(x)

    return x + res

In [130]:
def build_model(
    input_shape, head_size, num_heads, ff_dim, num_transformer_blocks,
    mlp_units, dropout=0, mlp_dropout=0
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)
    x = keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = keras.layers.Dense(dim, activation="relu")(x)
        x = keras.layers.Dropout(mlp_dropout)(x)
    outputs = keras.layers.Dense(len(classes), activation="softmax")(x)

    return keras.Model(inputs, outputs)

In [131]:
input_shape = xtr.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["sparse_categorical_accuracy"]
)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 500, 1)]     0           []                               
                                                                                                  
 multi_head_attention (MultiHea  (None, 500, 1)      7169        ['input_1[0][0]',                
 dAttention)                                                      'input_1[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 500, 1)       0           ['multi_head_attention[0][0]']   
                                                                                                  
 layer_normalization (LayerNorm  (None, 500, 1)      2           ['dropout[0][0]']            

In [132]:
epochs = 30
batch_size = 32

callbacks = [
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, verbose=1),
]

history = model.fit(
    xtr, ytr,
    batch_size=batch_size,
    epochs=epochs,
    callbacks=callbacks,
    validation_split=0.2,
    verbose=1
)

Epoch 1/30
90/90 [==============================] - 283s 3s/step - loss: 0.9519 - sparse_categorical_accuracy: 0.5257 - val_loss: 0.7054 - val_sparse_categorical_accuracy: 0.6006
Epoch 2/30
90/90 [==============================] - 278s 3s/step - loss: 0.8241 - sparse_categorical_accuracy: 0.5743 - val_loss: 0.6249 - val_sparse_categorical_accuracy: 0.6644
Epoch 3/30
90/90 [==============================] - 279s 3s/step - loss: 0.7464 - sparse_categorical_accuracy: 0.6247 - val_loss: 0.5854 - val_sparse_categorical_accuracy: 0.6865
Epoch 4/30
90/90 [==============================] - 284s 3s/step - loss: 0.7115 - sparse_categorical_accuracy: 0.6271 - val_loss: 0.5604 - val_sparse_categorical_accuracy: 0.7060
Epoch 5/30
90/90 [==============================] - 293s 3s/step - loss: 0.6671 - sparse_categorical_accuracy: 0.6674 - val_loss: 0.5434 - val_sparse_categorical_accuracy: 0.7240
Epoch 6/30
90/90 [==============================] - 292s 3s/step - loss: 0.6276 - sparse_categorical_accu

In [ ]:
metric = "sparse_categorical_accuracy"
history.history.keys()
plt.plot(history.history[metric])
plt.plot(history.history["val_"+metric])
plt.legend(["train", "val"])
plt.show()

In [137]:
model.evaluate(xts, yts, verbose=1)

42/42 [==============================] - 55s 1s/step - loss: 0.4287 - sparse_categorical_accuracy: 0.8121


[0.42870163917541504, 0.8121212124824524]